<a href="https://colab.research.google.com/github/antbartash/australian_rain/blob/main/BayesianOptimization_skopt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 1.4 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, cross_validate
from skopt.space import *
from skopt import BayesSearchCV

In [3]:
data = pd.read_csv(
    'https://raw.githubusercontent.com/antbartash/australian_rain/main/data/data_transformed.csv',
    index_col=0
  )

print(data.shape)
data.head()

(142193, 23)


,Location,WindGustDir,WindDir9am,WindDir3pm,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Month
0,2.0,13.0,13.0,14.0,13.4,22.9,0.6,NaN,NaN,44.0,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,0.0,0.0,12.0
1,2.0,14.0,6.0,15.0,7.4,25.1,0.0,NaN,NaN,44.0,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,0.0,0.0,12.0
2,2.0,15.0,13.0,15.0,12.9,25.7,0.0,NaN,NaN,46.0,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,0.0,0.0,12.0
3,2.0,4.0,9.0,0.0,9.2,28.0,0.0,NaN,NaN,24.0,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,0.0,0.0,12.0
4,2.0,13.0,1.0,7.0,17.5,32.3,1.0,NaN,NaN,41.0,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0.0,0.0,12.0


In [4]:
X, y = data.drop(columns=['RainTomorrow', 'RainToday']), data['RainTomorrow']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

In [5]:
def change_dtypes(data):
  for column in ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']:
    data[column] = data[column].astype('category')
  return data

X_train = change_dtypes(X_train)
X_test = change_dtypes(X_test)

X_train.dtypes

Location         category
WindGustDir      category
WindDir9am       category
WindDir3pm       category
MinTemp           float64
MaxTemp           float64
Rainfall          float64
Evaporation       float64
Sunshine          float64
WindGustSpeed     float64
WindSpeed9am      float64
WindSpeed3pm      float64
Humidity9am       float64
Humidity3pm       float64
Pressure9am       float64
Pressure3pm       float64
Cloud9am          float64
Cloud3pm          float64
Temp9am           float64
Temp3pm           float64
Month             float64
dtype: object

# Baseline model

In [6]:
baseline_lgbm = LGBMClassifier(random_state=42, verbose=0)
cv_result = cross_validate(baseline_lgbm, X, y, scoring='roc_auc',
                           return_train_score=True)

print(f"Mean train Gini: {cv_result['train_score'].mean() * 2 - 1}")
print(f"Mean test Gini: {cv_result['test_score'].mean() * 2 - 1}")
print(f"Mean fit time: {np.round(cv_result['fit_time'].mean(), 2)} s")

Mean train Gini: 0.8193700395010979
Mean test Gini: 0.7007491769077299
Mean fit time: 6.5 s


# Bayesian Optimization Gaussian Process (BOGP)

In [ ]:
hyperparameter_space = {
    'n_estimators': Integer(low=100, high=10000),
    'learning_rate': Real(low=1e-6, high=0.3, prior='log-uniform'),
    'num_leaves': Integer(10, 250),
    'max_depth': Integer(1, 50),
    'class_weight': Categorical([None, 'balanced']),
    'colsample_bytree': Real(0.3, 1.0),
    'reg_alpha': Real(0, 100),
    'reg_lambda': Real(0, 100)
}

model = LGBMClassifier(random_state=42, verbose=0)

clf = BayesSearchCV(
    model, hyperparameter_space, n_iter=50,
    optimizer_kwargs={
        'base_estimator': 'GP',
        'n_initial_points': 10,
        'initial_point_generator': 'random',
        'acq_func': 'gp_hedge', # probabilistically choose one of ['EI', 'LCB', 'PI'] acquisition functions at every iteration
        'acq_func_kwargs': {'xi': 0.01},
        'n_jobs': -1, # default: 1
        'random_state': 42
    },
    scoring='roc_auc', cv=3,
    random_state=42, n_jobs=-1, verbose=False
)

clf.fit(X_train, y_train)
print(f"Best Gini: {clf.best_score_ * 2 - 1}")
print(f"Best params: {clf.best_params_}")